In [72]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pytz


In [2]:

# google search headline scraper
def scrape_google_headlines(query):
    url = f"https://www.google.com/search?q={query}&hl=en"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        headlines = []
        for headline in soup.find_all('h3'):
            headlines.append(headline.text)
        return headlines
    else:
        print("Failed to retrieve search results.")
        return []

query = "Sports yesterday"
headlines = scrape_google_headlines(query)
if headlines:
    print("Headlines for the search query:", query)
    for idx, headline in enumerate(headlines, 1):
        print(f"{idx}. {headline}")
else:
    print("No headlines found.")


Headlines for the search query: Sports yesterday
1. Today's Top Sports Scores and Games (All Sports)
2. Latest Sports News
3. Yahoo Sports: News, Scores, Video, Fantasy Games ...
4. FOX Sports News, Scores, Schedules, Odds, Shows, Streams ...
5. Latest sports news, videos, interviews and comment
6. Sport Scores, Stats and Highlights
7. Sports News, Scores, Predictions and Analysis - USA TODAY
8. All Sports Scoreboard
9. CBS Sports - News, Live Scores, Schedules, Fantasy Games ...
10. More results
11. Try again


In [ ]:
def is_yesterday(date_string):
    # Define the format of the input date string without the timezone part
    date_format = '%B %d, %Y, %I:%M %p ET'
    
    # Parse the date string into a naive datetime object
    naive_date_time = datetime.strptime(date_string, date_format)
    
    # Define the timezone (Eastern Time)
    eastern = pytz.timezone('US/Eastern')
    
    # Localize the naive datetime to Eastern Time
    eastern_date_time = eastern.localize(naive_date_time)
    
    # Get the current date and time in Eastern Time
    now_eastern = datetime.now(pytz.timezone('US/Eastern'))
    
    # Calculate the date for yesterday in Eastern Time
    yesterday_eastern = now_eastern - timedelta(days=1)
    
    # Check if the date is exactly yesterday (ignoring time)
    return eastern_date_time.date() == yesterday_eastern.date()

In [3]:

url = "https://www.espn.com/espn/latestnews"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

In [82]:
articles = soup.find(class_="span-4")
sports = articles.find_all('h3')
sport_headlines_dict = {}
for sport in sports:
    sport = sport.text
    sport_headlines = soup.find('h3', string=sport).find_next_sibling().find_all('li')
    sport_headlines_dict[sport] = []
    for headline in sport_headlines:
        htext = headline.get_text(strip=True)
        title = htext[0:htext.rfind('(')]
        date = htext[htext.rfind('(')+1:-1]
        if is_yesterday(date):
            sport_headlines_dict[sport].append(title)


In [ ]:
#this is good, but I think im scraping wrong thing, headlines may not be the way to go, maybe read the top article for each in season sport on ESPN homepage